# Programmatic debugging and dissection of ML models with NoPdb

[NoPdb](https://github.com/cifkao/nopdb) is the first non-interactive (programmatic) debugger for Python. It provides a user-friendly API for debugger-like features in the form of convenient context managers. The current version supports:
- capturing arguments, local variables, return values and stack traces of function calls;
- capturing the value of a given expression at a given "breakpoint";
- temporary, on-the-fly injection of arbitrary code into existing functions.

NoPdb is available under the BSD 3-Clause license and installable from PyPI via `pip install nopdb`.

In [1]:
import nopdb

## NoPdb features

### Capturing function calls

The functions `capture_call()` and `capture_calls()` allow capturing useful information about calls to a given function.
They are typically used as context managers, e.g.:

``` python
with nopdb.capture_call(fn) as call:
    some_code_that_calls_fn()

    print(call)  # See details about how fn()
                 # was called
```

For a more concrete example, let us define two simple functions:

In [2]:
def f(x, y):
    z = x + y
    return 2 * z

def g(x):
    return f(x, x)

We now wish to call `g()` while capturing information about the call to `f()` within it:

In [4]:
with nopdb.capture_call(f) as call:
    print(g(1))

4


The variable `call` now holds the captured arguments, local variables, return value and stack trace of the call:

In [5]:
call

CallCapture(name='f', args=OrderedDict(x=1, y=1), return_value=4)

In [18]:
call.args['x'], call.args['y'], call.return_value

(1, 1, 4)

In [8]:
call.locals

{'x': 1, 'y': 1, 'z': 2}

In [9]:
call.print_stack()

  File "/srv/conda/envs/notebook/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/srv/conda/envs/notebook/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/srv/conda/envs/notebook/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
    self._run_once()
  File "/srv/conda/envs/notebook/lib/python3.7/asyncio/base_events.py", line 1786, in _run_once
    handl

If `f()` was called multiple times within the context manager block,
the variable `call` would only store the *most recent* call.
To capture *all* calls, one can use `capture_calls()` (in the plural):

In [10]:
with nopdb.capture_calls(f) as calls:
    print(g(1))
    print(g(42))

4
168


In [11]:
calls

[CallInfo(name='f', args=OrderedDict(x=1, y=1), return_value=4),
 CallInfo(name='f', args=OrderedDict(x=42, y=42), return_value=168)]

Both `capture_call` and `capture_calls` support different ways of
specifying which function(s) should be considered:
- We may pass a function or its name, i.e. `capture_calls(f)` or
  `capture_calls('f')`.
- Passing a method bound to an instance, as in `capture_calls(obj.f)`,
  will work as expected: only calls invoked on that particular instance (and
  not other instances of the same class) will be captured.
- A module, a filename or a full file path can be passed, e.g.
  `capture_calls('f', module=mymodule)`
  or
  `capture_calls('f', file='mymodule.py')`.

### Setting breakpoints

In a conventional debugger, a *breakpoint* pauses the
execution of the program and launches an interactive
debugging session.
In NoPdb, a *non-interactive* debugger, a breakpoint
instead triggers some actions defined in advance,
like evaluating expressions.

To set a breakpoint, call the `breakpoint` function. A breakpoint object
is returned, allowing to define actions using its methods
`eval()` (evaluate an expression),
`exec()` (execute a statement) and
`debug()` (enter an interactive debugger).

#### Evaluating expressions

Using the example from the previous section, let us
try to use a breakpoint to capture
the value of a variable:

In [12]:
with nopdb.breakpoint(f, line=3) as bp:
    # Get the value of z whenever
    # the breakpoint is hit
    z_values = bp.eval('z')  

    print(g(1))
    print(g(42))

4
168


In [13]:
z_values

[2, 84]

#### Executing statements

Unlike some conventional debuggers, NoPdb allows
executing arbitrary statements including variable
assignments (note that assigning to local variables is an experimental feature and is only supported under some Python implementations):

In [15]:
with nopdb.breakpoint(f, line=3) as bp:
    # Get the value of z, then increment it,
    # then get the new value
    z_before = bp.eval('z')
    bp.exec('z += 1')
    z_after = bp.eval('z')

    print(g(1))  # This would normally print 4

6


In [16]:
z_before, z_after

([2], [3])

## Application: visualizing ML model features